In [1]:
import numpy as np

In [2]:
def softmax_stable(Z):
    # each row of Z is a set of score
    e_Z = np.exp(Z - np.max(Z, axis = 1, keepdims=True))
    A = e_Z / e_Z.sum(axis = 1, keepdims = True)
    return A

In [3]:
def crossentropy_loss(Yhat, y):
    """
    Yhat: a numpy array of shape (Npoints, nClasses) -- predict output
    y: a numpy array of shape (Npoints) -- ground truth.
    We dont need to use one-hot display of y here since most of element are zeros. Each row of Yhat, we just need to access to the corresponding index only.
    """
    return -np.mean(np.log(Yhat[range(Yhat.shape[0]), y]))

In [5]:
def MLP_init(d0, d1, d2): # depend on number of nodes in each layer of MLP
    """
    Initialize W1, b1, W2, b2
    d0: dimention of input data
    d1: number of hiden layer's units
    d2: number of output units = number of classes
    """
    W1 = 0.01 * np.random.randn(d0, d1)
    b1 = np.zeros(d1)
    W2 = 0.01 * np.random.randn(d1, d2)
    b2 = np.zeros(d2)
    return (W1, b1, W2, b2)

In [4]:
def MLP_predict(X, W1, b1, W2, b2):
    """
    Suppose the network has been trained, predict class of new points.
    X: data matrix, each Row is one data point
    W1, b1, W2, b2: learned weight matrices and biases
    """
    Z1 = X.dot(W1) + b1
    # Z1 shape of (N, d1)
    # W1 shape of (d0, d1)
    # b1 shape of (d1,)
    # X shape of (N, d0)
    # note that we dont need to transpose the W1 matrix.
    A1 = np.maximum(Z1, 0)
    # A1 shape of (N, d1)
    Z2 = A1.dot(W2) + b2
    # W2 shape of (d1, d2)
    # b2 shape of (d2,)
    # note that we dont need to transpose the W2 matrix
    # Z2 shape of (N, d2)
    Yhat = A2 = softmax_stable(Z2)
    # A2, Yhat shape of (N, d2) because softmax_stable didnt change the shape of those.
    return np.argmax(Z2, axis=1)

In [11]:
def MLP_fit(X, y, W1, b1, W2, b2, eta, batch_size = 30):
    # eta is learning rate
    # X shape of (N, d0)
    # y shape of nPoints
    lost_hist = []
    N = X.shape[0]
    nbatches = int(np.ceil(float(N)/batch_size))
    for ep in range(20000): # number of epochs
        mix_idxs = np.random.permutation(N) # stochastic
        
        for i in range (nbatches):
            batch_idxs = mix_idxs[batch_size*i : min(batch_size*(i+1), N)]
            X_batch, y_batch = X[batch_idxs], y[batch_idxs]
            # Feed-Forward
            Z1 = X_batch.dot(W1) + b1
            # Z1 shape of (N, d1)
            A1 = np.maximum(Z1, 0)
            # A1 shape of (N, d1)
            Z2 = A1.dot(W2) + b2
            # Z2 shape of (N, d2)
            Yhat = softmax_stable(Z2)
            # Yhat shape of (N, d2)

            # print lost value after each 1000 epochs
            if ep % 1000 == 0 and i == nbatches - 1:
                loss = crossentropy_loss(Yhat, y_batch)
                print("iter %d, loss: %f" % (ep, loss))
                lost_hist.append(loss)

            # backpropagation batch
            id0 = range(Yhat.shape[0])
            Yhat[id0, y_batch] -= 1
            E2 = Yhat / X.shape[0]
            # E2 shape of (N, d2)
            gW2 = A1.T.dot(E2)
            # gW2 shape of (d1, d2)
            gb2 = np.sum(E2, axis = 0)
            # gb2 shape of (d2,)
            E1 = E2.dot(W2.T)
            E1[Z1 <= 0] = 0
            # E1 shape of (N, d1)
            # derivative of ReLU (Z1) = 0 when e_{i} in Z1 smaller than 0, and = 1 when e_{i} in Z1 greater than 0.
            gW1 = X_batch.T.dot(E1)
            gb1 = np.sum(E1, axis = 0)
            # gW1 shape of (d0, d1)

            # Gradient Descent
            W1 += -eta * gW1
            W2 += -eta * gW2
            b1 += -eta * gb1
            b2 += -eta * gb2
    return W1, b1, W2, b2, lost_hist

In [13]:
means = [[-1, -1], [1, -1], [0, 1]]
cov = [[1, 0], [0, 1]]
N = 200
X0 = np.random.multivariate_normal(means[0], cov, N)
X1 = np.random.multivariate_normal(means[1], cov, N)
X2 = np.random.multivariate_normal(means[2], cov, N)
X = np.concatenate((X0, X1, X2), axis = 0)
y = np.asarray([0]*N + [1]*N + [2]*N)

# Supose X, y are training input and output, respectively
d0 = 2  # data dimention
d1 = h = 2000 # number of hidden units
d2 = C = 3 # number of class in last layer
eta = 1 # learning rate
(W1, b1, W2, b2) = MLP_init(d0, d1, d2)
(W1, b1, W2, b2, lost_hist) = MLP_fit(X, y, W1, b1, W2, b2, eta)
y_pred = MLP_predict(X, W1, b1, W2, b2 )
acc = 100 * np.mean(y_pred == y)
print('tranning accuracy: %.2f %%' % acc)

iter 0, loss: 1.035338
iter 1000, loss: 0.349936
iter 2000, loss: 0.439657
iter 3000, loss: 0.538261
iter 4000, loss: 0.435431
iter 5000, loss: 0.475841
iter 6000, loss: 0.486963
iter 7000, loss: 0.530784
iter 8000, loss: 0.514993
iter 9000, loss: 0.644424
iter 10000, loss: 0.343045
iter 11000, loss: 0.622766
iter 12000, loss: 0.379253
iter 13000, loss: 0.519054
iter 14000, loss: 0.340559
iter 15000, loss: 0.301515
iter 16000, loss: 0.718444
iter 17000, loss: 0.431808
iter 18000, loss: 0.445064
iter 19000, loss: 0.309719
tranning accuracy: 80.67 %
